#### Use this for sentiment analysis with Logistic Regression and Naive Bayes
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/1_IMPORTANT_Capstone-Project-master/Capstone-Project-master/Capstone_Project_Sentiment_Analysis.ipynb
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/1_IMPORTANT_Capstone-Project-master/Capstone-Project-master/Capstone_Project_KNN_RecommenderSystem.ipynb
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/2_Amazon-Product-Recommender-System-master/Naive%20Bayes/Amazon%20reviews_Naive%20Bayes.ipynb
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/2_Amazon-Product-Recommender-System-master/Recommender%20System/Recommender%20System.ipynb

Removing Contractions - https://medium.com/analytics-vidhya/sentiment-analysis-on-amazon-reviews-using-tf-idf-approach-c5ab4c36e7a1


http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/Recommendation%2BSystem%2BNotebook_WIP.ipynb
http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/Recommendation%2BSystem%2BNotebook.ipynb

http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/Akash_collaborative_user_base..ipynb

In [154]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

In [155]:
import re

In [156]:
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn import neighbors
import matplotlib.pyplot as plt


In [157]:
df = pd.read_csv("sample30.csv")

In [158]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [159]:
print(df.columns)
print(df.shape)

Index(['id', 'brand', 'categories', 'manufacturer', 'name', 'reviews_date',
       'reviews_didPurchase', 'reviews_doRecommend', 'reviews_rating',
       'reviews_text', 'reviews_title', 'reviews_userCity',
       'reviews_userProvince', 'reviews_username', 'user_sentiment'],
      dtype='object')
(30000, 15)


In [160]:
df.head()

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,reviews_username,user_sentiment
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30T06:21:45.000Z,NaN,NaN,5,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,joshua,Positive
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor. This review was collected as part...,Good,NaN,NaN,dorothy w,Positive
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor.,Good,NaN,NaN,dorothy w,Positive
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06T00:00:00.000Z,False,False,1,I read through the reviews on here before look...,Disappointed,NaN,NaN,rebecca,Negative
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21T00:00:00.000Z,False,False,1,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,walker557,Negative


In [161]:
#Tokenize the review text to words
reg = re.compile('[^a-z]+')
def tokenizeReview(text):
    text = text.lower()
    text = reg.sub(' ', text).strip()
    return text

In [162]:
df['reviews_tokenized'] = df['reviews_text'].apply(tokenizeReview)

In [163]:
df.shape

(30000, 16)

In [164]:
df = df.drop(['reviews_text'], axis=1)

In [165]:
df.shape

(30000, 15)

In [166]:
df['reviews_tokenized'].head(5)

0    i love this album it s very good more to the h...
1    good flavor this review was collected as part ...
2                                          good flavor
3    i read through the reviews on here before look...
4    my husband bought this gel for us the gel caus...
Name: reviews_tokenized, dtype: object

In [167]:
df['user_sentiment'].value_counts()

Positive    26632
Negative     3367
Name: user_sentiment, dtype: int64

In [168]:
def senti(txt):
    if str(txt).lower() == 'positive':
        return 1
    else:
        return 0
    
X = df['reviews_tokenized']
y = df['user_sentiment'].apply(senti)

In [169]:
print(len(X))
print(len(y))

30000
30000


In [170]:
y

0        1
1        1
2        1
3        0
4        0
        ..
29995    1
29996    1
29997    1
29998    1
29999    1
Name: user_sentiment, Length: 30000, dtype: int64

In [171]:
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word', stop_words= 'english')

In [172]:
def model_fit(X, y, feature_model,ml_model,coef_show=1):
    
    X_features = feature_model.fit_transform(X)
    print('# features: {}'.format(X_features.shape[1]))
    
#    tempX = feature_model.transform(['this is an awful product', 'i am satisfied with this. will come back again'])
    
    X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size = 0.2, random_state=43)
    clf = ml_model.fit(X_train, y_train)
    clf_pred = clf.predict(X_test)
    accuracy = clf.score(X_test, y_test)
    model_performance = classification_report(y_test, clf_pred)
    print ('accuracy of the model: ', accuracy)
    print('')
    print(model_performance)

#    print(X_train)
#    print("---")
#    print(X_test)
#    print("---")
#    print(dir(ml_model))
    
#    tempPred = ml_model.predict(tempX)
#    print('---Prediction----')
#    print(tempPred)
    
#    print(vars(X_test))
#    print("indices - ", len(X_test.indices))
#    print("indptr - ", len(X_test.indptr))
#    print("data - ", len(X_test.data))
    
#    print(dir(X_test))
#    print("X_ARRAY ", X_test._get_arrayXarray(0, 529))
    
    
    if coef_show == 1: 
        w = feature_model.get_feature_names()
        coef = clf.coef_.tolist()[0]
        coeff_df = pd.DataFrame({'Word' : w, 'Coefficient' : coef})
        coeff_df = coeff_df.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
        print('')
        print('Top 10 positive features (variables)')
        print(coeff_df.head(20).to_string(index=False))
        print('')
        print('Top 10 negative features (variables)')        
        print(coeff_df.tail(20).to_string(index=False))
       
    return X_test, clf_pred

xtest, pred = model_fit(X, y, tfidf,LogisticRegression(),coef_show=1)

# features: 1000
accuracy of the model:  0.9221666666666667

              precision    recall  f1-score   support

           0       0.81      0.38      0.52       660
           1       0.93      0.99      0.96      5340

    accuracy                           0.92      6000
   macro avg       0.87      0.69      0.74      6000
weighted avg       0.92      0.92      0.91      6000


Top 10 positive features (variables)
      Word  Coefficient
     great    13.299884
      love    11.054383
      best     8.156580
      good     7.800511
      easy     7.735733
     clean     7.719841
    better     5.808192
     loved     5.688351
      nice     5.268984
 excellent     5.121848
   awesome     4.898837
   perfect     4.861898
  favorite     4.555234
   enjoyed     4.373794
   amazing     4.364436
      free     3.939587
 wonderful     3.916948
    really     3.575024
     handy     3.529437
     happy     3.373818

Top 10 negative features (variables)
          Word  Coefficient
    

In [173]:
xtest, pred = model_fit(X, y, tfidf,LogisticRegression(),coef_show=1)

# features: 1000
accuracy of the model:  0.9221666666666667

              precision    recall  f1-score   support

           0       0.81      0.38      0.52       660
           1       0.93      0.99      0.96      5340

    accuracy                           0.92      6000
   macro avg       0.87      0.69      0.74      6000
weighted avg       0.92      0.92      0.91      6000


Top 10 positive features (variables)
      Word  Coefficient
     great    13.299884
      love    11.054383
      best     8.156580
      good     7.800511
      easy     7.735733
     clean     7.719841
    better     5.808192
     loved     5.688351
      nice     5.268984
 excellent     5.121848
   awesome     4.898837
   perfect     4.861898
  favorite     4.555234
   enjoyed     4.373794
   amazing     4.364436
      free     3.939587
 wonderful     3.916948
    really     3.575024
     handy     3.529437
     happy     3.373818

Top 10 negative features (variables)
          Word  Coefficient
    

In [174]:
print(xtest)
print(pred) 

  (0, 529)	0.22241595149271842
  (0, 47)	0.20964319801245335
  (0, 720)	0.696980923623149
  (0, 771)	0.21593130586186834
  (0, 90)	0.40026789622812714
  (0, 969)	0.1885671404031856
  (0, 371)	0.16567427089889267
  (0, 511)	0.1772086562204112
  (0, 940)	0.1333247534125354
  (0, 619)	0.20415080720516499
  (0, 61)	0.11974647057071268
  (0, 343)	0.13457303762618536
  (0, 832)	0.16627431734861037
  (1, 569)	0.24218436278559
  (1, 154)	0.6098511233084832
  (1, 526)	0.4614886748852459
  (1, 619)	0.5597691993889182
  (1, 524)	0.20764148956715778
  (2, 902)	0.6130275606318966
  (2, 859)	0.6486652086683559
  (2, 677)	0.22152244726222683
  (2, 147)	0.22152244726222683
  (2, 718)	0.22001360570007655
  (2, 524)	0.23849587206847778
  (3, 110)	0.5440276702573296
  :	:
  (5998, 3)	0.2557293834011859
  (5998, 110)	0.32355073600328416
  (5998, 570)	0.23649604270272198
  (5998, 283)	0.36865793862816465
  (5998, 752)	0.3047600169042265
  (5998, 940)	0.23511843058393603
  (5998, 155)	0.28812486505187546
  

#### Recommendation System - User based collaborative

In [175]:
# Reading ratings file
ratings = pd.read_csv('sample30.csv' , encoding='latin-1')
ratings.head()

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,reviews_username,user_sentiment
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30T06:21:45.000Z,NaN,NaN,5,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,joshua,Positive
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor. This review was collected as part...,Good,NaN,NaN,dorothy w,Positive
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor.,Good,NaN,NaN,dorothy w,Positive
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06T00:00:00.000Z,False,False,1,I read through the reviews on here before look...,Disappointed,NaN,NaN,rebecca,Negative
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21T00:00:00.000Z,False,False,1,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,walker557,Negative


In [176]:
ratings = ratings[['id', 'reviews_username', 'name', 'reviews_rating']]
ratings.head()

,id,reviews_username,name,reviews_rating
0,AV13O1A8GV-KLJ3akUyj,joshua,Pink Friday: Roman Reloaded Re-Up (w/dvd),5
1,AV14LG0R-jtxr-f38QfS,dorothy w,Lundberg Organic Cinnamon Toast Rice Cakes,5
2,AV14LG0R-jtxr-f38QfS,dorothy w,Lundberg Organic Cinnamon Toast Rice Cakes,5
3,AV16khLE-jtxr-f38VFn,rebecca,K-Y Love Sensuality Pleasure Gel,1
4,AV16khLE-jtxr-f38VFn,walker557,K-Y Love Sensuality Pleasure Gel,1


In [177]:
ratings.reviews_rating.unique()

array([5, 1, 3, 4, 2], dtype=int64)

In [178]:
#Number of users
lst = ratings.reviews_username.value_counts()
print(lst)
print(len(lst))
print(sum(lst))

mike                 41
byamazon customer    41
chris                32
lisa                 16
tony                 15
                     ..
daveg                 1
watchman              1
cala                  1
neese_xo              1
em625                 1
Name: reviews_username, Length: 24914, dtype: int64
24914
29937


In [179]:
ratings.reviews_rating.value_counts()

5    20831
4     6020
1     1384
3     1345
2      420
Name: reviews_rating, dtype: int64

In [180]:
print(ratings.shape)
print(ratings.id.value_counts())

(30000, 4)
AVpf3VOfilAPnD_xjpun    8545
AVpfPaoqLJeJML435Xk9    3325
AVpfJP1C1cnluZ0-e3Xy    2039
AVpfW8y_LJeJML437ySW    1186
AVpfRTh1ilAPnD_xYic2    1143
                        ... 
AV2Avn5dGV-KLJ3alTjq       1
AVpfDflP1cnluZ0-c5RJ       1
AV1Yo6FPglJLPUi8IK3u       1
AVpfRxSkilAPnD_xYrzm       1
AVpf8AoBilAPnD_xkzJ2       1
Name: id, Length: 271, dtype: int64


### Dividing the dataset into train and test

In [181]:
# Test and Train split of the dataset.
from sklearn.model_selection import train_test_split
train, test = train_test_split(ratings, test_size=0.30, random_state=31)

In [182]:
print(train.shape)
print(test.shape)

(21000, 4)
(9000, 4)


In [183]:
#Proof that the default aggregate function for duplicates is mean
d = pd.read_csv('TestFile.csv' , encoding='latin-1')
print(d)
t = d.pivot_table(index='reviews_username', columns='name', values='reviews_rating')
t

   id   name  reviews_rating reviews_username
0   1  brush               3             shal
1   2  brush               4             shal
2   3    pen               5             shal
3   4    pen               5            kumar
4   5  paint               4             shal
5   6  paint               2            kumar
6   7  board               1             shal
7   8  board               2            kumar
8   9  brush               3             shal


name,board,brush,paint,pen
reviews_username,,,,
kumar,2.0,NaN,2.0,5.0
shal,1.0,3.333333,4.0,5.0


In [184]:
print(len(ratings.reviews_username.unique()))
print(len(ratings.name.unique()))

tmp = ratings.pivot_table(index='reviews_username', columns='name', values='reviews_rating').fillna(0)
tmp.head()

24915
271


name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz","42 Dual Drop Leaf Table with 2 Madrid Chairs""",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),...,Walkers Stem Ginger Shortbread,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Wilton Black Dots Standard Baking Cups,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01impala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02dakota,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02deuce,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [185]:
# Pivot the train ratings' dataset into matrix format in which columns are items and the rows are user.
df_pivot = train.pivot_table(index='reviews_username', columns='name', values='reviews_rating').fillna(0)
df_pivot.head(5)

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01impala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0325home,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06stidriver,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Creating dummy train & dummy test dataset
These dataset will be used for prediction 
- Dummy train will be used later for prediction of the movies which has not been rated by the user. To ignore the movies rated by the user, we will mark it as 0 during prediction. The movies not rated by user is marked as 1 for prediction in dummy train dataset. 

- Dummy test will be used for evaluation. To evaluate, we will only make prediction on the movies rated by the user. So, this is marked as 1. This is just opposite of dummy_train.

In [214]:
# Copy the train dataset into dummy_train
dummy_train = train.copy()
dummy_train.head()

,id,reviews_username,name,reviews_rating
11155,AVpf3VOfilAPnD_xjpun,gor05751,Clorox Disinfecting Wipes Value Pack Scented 1...,5
22114,AVpf--o9LJeJML43G_Fq,annavsl,Vaseline Intensive Care Healthy Hands Stronger...,4
26218,AVpfPkrkilAPnD_xX-L9,bluenose,"Finish Gelpacs Dishwasher Detergent, Orange Sc...",5
18966,AVpfJP1C1cnluZ0-e3Xy,kiki,Clorox Disinfecting Bathroom Cleaner,5
29999,AVpfW8y_LJeJML437ySW,kcoopxoxo,L'or233al Paris Elvive Extraordinary Clay Reba...,5


In [215]:
print(dummy_train['reviews_rating'].value_counts())
print(dummy_train.shape)

5    14514
4     4234
1      995
3      957
2      300
Name: reviews_rating, dtype: int64
(21000, 4)


In [216]:
# The movies not rated by user is marked as 1 for prediction. 
dummy_train['reviews_rating'] = dummy_train['reviews_rating'].apply(lambda x: 0 if x>=1 else 1)

In [217]:
dummy_train.reviews_rating.unique()

array([0], dtype=int64)

In [218]:
dummy_train.head()

,id,reviews_username,name,reviews_rating
11155,AVpf3VOfilAPnD_xjpun,gor05751,Clorox Disinfecting Wipes Value Pack Scented 1...,0
22114,AVpf--o9LJeJML43G_Fq,annavsl,Vaseline Intensive Care Healthy Hands Stronger...,0
26218,AVpfPkrkilAPnD_xX-L9,bluenose,"Finish Gelpacs Dishwasher Detergent, Orange Sc...",0
18966,AVpfJP1C1cnluZ0-e3Xy,kiki,Clorox Disinfecting Bathroom Cleaner,0
29999,AVpfW8y_LJeJML437ySW,kcoopxoxo,L'or233al Paris Elvive Extraordinary Clay Reba...,0


In [219]:
# Convert the dummy train dataset into matrix format.
dummy_train = dummy_train.pivot_table(index='reviews_username', columns='name', values='reviews_rating').fillna(1)

In [220]:
dummy_train.shape

(18252, 252)

In [221]:
dummy_train.loc['0325home'].value_counts()

1.0    251
0.0      1
Name: 0325home, dtype: int64

In [222]:
dummy_train

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
00sab00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
01impala,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
0325home,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
06stidriver,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zulaa118,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
zxcsdfd,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
zxjki,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [223]:
dummy_train.loc['06stidriver'].value_counts()

1.0    251
0.0      1
Name: 06stidriver, dtype: int64

**Cosine Similarity**

Cosine Similarity is a measurement that quantifies the similarity between two vectors [Which is Rating Vector in this case] 

**Adjusted Cosine**

Adjusted cosine similarity is a modified version of vector-based similarity where we incorporate the fact that different users have different ratings schemes. In other words, some users might rate items highly in general, and others might give items lower ratings as a preference. To handle this nature from rating given by user , we subtract average ratings for each user from each user's rating for different movies.

# User Similarity Matrix

## Using Cosine Similarity
#### Pair-wise similarity is done row-wise in the pivot table

In [224]:
from sklearn.metrics.pairwise import pairwise_distances

# Creating the User Similarity Matrix using pairwise_distance function.
user_correlation = 1 - pairwise_distances(df_pivot, metric='cosine')
print(user_correlation)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [235]:
print(sum(user_correlation[0])/len(user_correlation[0]))
print(sum(user_correlation[1])/len(user_correlation[1]))
print(sum(user_correlation[2])/len(user_correlation[2]))
print(sum(user_correlation[3])/len(user_correlation[3]))
print(sum(user_correlation[4])/len(user_correlation[4]))
print(sum(user_correlation[5])/len(user_correlation[5]))
print(sum(user_correlation[6])/len(user_correlation[6]))
print(sum(user_correlation[7])/len(user_correlation[7]))


0.0007699409750198169
0.025349981632427198
0.11795404456236924
0.2965389464248795
0.2965389464248795
0.06681913443692394
0.02378234824574021
0.003206839857539405


In [236]:
user_correlation[np.isnan(user_correlation)] = 0

In [237]:
print(user_correlation)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [238]:
user_correlation.shape

(18252, 18252)

## Using adjusted Cosine 

### Here, we are not removing the NaN values and calculating the mean only for the movies rated by the user

In [240]:
# Create a user-item review matrix.
df_pivot = train.pivot_table(index='reviews_username', columns='name', values='reviews_rating')

In [241]:
df_pivot.head()

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00sab00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01impala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0325home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
06stidriver,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Normalising the rating of the movie for each user around 0 mean

In [242]:
#This does the mean for each row as the axis is 1
mean = np.nanmean(df_pivot, axis=1)

In [243]:
print(mean)
print(len(mean))

[4. 1. 3. ... 2. 5. 5.]
18252


In [244]:
df_subtracted = (df_pivot.T - mean).T

In [245]:
df_subtracted.head()

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00sab00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01impala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0325home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
06stidriver,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Finding cosine similarity

In [246]:
from sklearn.metrics.pairwise import pairwise_distances

In [247]:
# Creating the User Similarity Matrix using pairwise_distance function.
user_correlation = 1 - pairwise_distances(df_subtracted.fillna(0), metric='cosine')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [255]:
print(sum(user_correlation[:][253]))

0.0


## Prediction - User User

Doing the prediction for the users which are positively related with other users, and not the users which are negatively related as we are interested in the users which are more similar to the current users. So, ignoring the correlation for values less than 0. 

In [256]:
user_correlation[user_correlation<0]=0
user_correlation

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Rating predicted by the user (for movies rated as well as not rated) is the weighted sum of correlation with the movie rating (as present in the rating dataset). 

In [258]:
sum(sum(user_correlation))

10185.884430507153

In [259]:
user_predicted_ratings = np.dot(user_correlation, df_pivot.fillna(0))
user_predicted_ratings

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [260]:
user_predicted_ratings.shape

(18252, 252)

In [271]:
sum(sum(user_predicted_ratings))

120324.65698632221

Since we are interested only in the movies not rated by the user, we will ignore the movies rated by the user by making it zero. 

In [261]:
user_final_rating = np.multiply(user_predicted_ratings,dummy_train)
user_final_rating.head()

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01impala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0325home,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06stidriver,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [292]:
print(user_final_rating.T.sum(axis=1).sort_values(ascending=False)[0:60])

name
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)                                        5404.701432
The Resident Evil Collection 5 Discs (blu-Ray)                                                             5196.472401
Mike Dave Need Wedding Dates (dvd + Digital)                                                               5117.180772
Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd                                            4756.145677
My Big Fat Greek Wedding 2 (blu-Ray + Dvd + Digital)                                                       4343.492836
Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total                                                  4053.226287
Red (special Edition) (dvdvideo)                                                                           2982.062401
Clorox Disinfecting Bathroom Cleaner                                                                       2413.224154
Windex Original Glass Cleaner Refill 67.6oz

In [281]:
d = user_final_rating.loc['sheila'].sort_values(ascending=False)[0:10]
d

name
Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd                    15.413725
Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total                           6.428598
My Big Fat Greek Wedding 2 (blu-Ray + Dvd + Digital)                                5.306753
Coty Airspun Face Powder, Translucent Extra Coverage                                4.854885
The Resident Evil Collection 5 Discs (blu-Ray)                                      3.696566
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)                 2.842987
Windex Original Glass Cleaner Refill 67.6oz (2 Liter)                               2.556805
Tostitos Bite Size Tortilla Chips                                                   2.474985
Pendaflex174 Divide It Up File Folder, Multi Section, Letter, Assorted, 12/pack     2.415863
There's Something About Mary (dvd)                                                  1.996316
Name: sheila, dtype: float64